In [13]:
import h5py
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import LSTM
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
#loading the weights from the saved file but for that we have to compile the model
#Compiling the model
vocabsize=9400
vector_embedding = 128
maxlen = 4
fixlen=3
model = Sequential()
model.add(LSTM(input_dim=vocabsize+1, output_dim=vector_embedding, input_length=fixlen, return_sequences=True))
model.add(Activation('sigmoid'))
model.add(Dropout(0.2))
model.add(LSTM(input_dim=vector_embedding, output_dim=vector_embedding, input_length=fixlen, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(vocabsize+1))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#Loading the weights
directory ='/home/sinah/'
weights_path= directory + 'model_weight.hdf5'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k ==0:
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
        x=model.layers[k].get_weights()
        embedding = np.array(x[0])
    else:
        break
f.close()
print('Embedding Weights loaded.')

#Sigmoid transformation of weights and then to binary
top_words_count=50
embedding = 1/(1+np.exp(embedding*-1))
embedding_capped_with_max_words=np.zeros((top_words_count,len(embedding[0])))
for rows in range(0,top_words_count):
    for cols in range(0,len(embedding[0])):
        if embedding[rows][cols]<.5:
            embedding_capped_with_max_words[rows][cols] = 0
        else:
            embedding_capped_with_max_words[rows][cols] = 1 

#Get the top 100 words in vocab  in word list
counter=0
text_open=open(directory + 'vocab.txt','r')
words=[]
for item in text_open:
    if counter < top_words_count:
        words.append(item.strip('\n'))
        counter = counter +1
    else:
        break

#Get TSNE representation of the words
model_tsne = TSNE(n_components=2, verbose=2, n_iter=200)
vis_data = model_tsne.fit_transform(embedding_capped_with_max_words)
print('vis_data  vector has shape', vis_data.shape)
# plot the result
vis_x = vis_data[:, 0]
vis_y = vis_data[:, 1]
fig,ax = plt.subplots()
ax.scatter(vis_x, vis_y)
for i, txt in enumerate(words):
    ax.annotate(txt, (vis_x[i],vis_y[i]))
plt.show()


Embedding Weights loaded.
[t-SNE] Computing pairwise distances...
[t-SNE] Computing 49 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 50 / 50
[t-SNE] Mean sigma: 2.252086
[t-SNE] Iteration 25: error = 1.6078917, gradient norm = 0.0027273
[t-SNE] Iteration 50: error = 1.5210387, gradient norm = 0.0019067
[t-SNE] Iteration 75: error = 1.4069742, gradient norm = 0.0009577
[t-SNE] Iteration 75: gradient norm 0.000958. Finished.
[t-SNE] Error after 75 iterations with early exaggeration: 1.406974
[t-SNE] Iteration 100: error = 1.3574212, gradient norm = 0.0008080
[t-SNE] Iteration 100: gradient norm 0.000808. Finished.
[t-SNE] Error after 100 iterations: 1.357421
('vis_data  vector has shape', (50, 2))
